<a href="https://colab.research.google.com/github/tomonari-masada/course2021-sml/blob/main/11_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 日本語の単語ベクトルをクラスタリング

* 使用するデータは下記の場所にあるもの

 * https://github.com/shiroyagicorp/japanese-word2vec-model-builder

* word2vecの技術を使って、日本語の単語が50次元のベクトルで表現されている。
 * word2vecそのものについては、ここでは説明しません。
* 今回はこの単語ベクトルをクラスタリングして、意味の近い単語が同じクラスタに属しているかをチェックする。

## 1) wgetコマンドでダウンロードし、unzipで解凍する

In [ ]:
!if [ ! -e latest-ja-word2vec-gensim-model.zip ]; then wget http://public.shiroyagi.s3.amazonaws.com/latest-ja-word2vec-gensim-model.zip ; fi

In [ ]:
!unzip -u latest-ja-word2vec-gensim-model.zip
#!unzip -u /content/drive/MyDrive/2021Courses/SML/latest-ja-word2vec-gensim-model.zip

## 2) ライブラリgensimを使ってデータを読み込む

In [ ]:
from gensim.models.word2vec import Word2Vec

model_path = 'word2vec.gensim.model'
model = Word2Vec.load(model_path)

* 単語ベクトルの次元を確認する



In [ ]:
print(model.vector_size)

* 単語ベクトルデータを変数名wvで参照することにする


In [ ]:
wv = model.wv

* 単語リストと、対応する単語ベクトルのリストを、作成する

In [ ]:
words = list()
vectors = list()
for word in wv.vocab:
  words.append(word)
  vectors.append(wv.word_vec(word))

* 単語数を確認する



In [ ]:
print(len(words))

* 最初の単語とその単語ベクトルを確認する



In [ ]:
print(words[0])
print(vectors[0])

## 3) NumPyの配列に変換する

In [ ]:
import numpy as np

words = np.array(words)
vectors = np.array(vectors)

In [ ]:
print(words)

In [ ]:
print(vectors)

* 「日本」という単語に最も近い10個の単語を表示させてみる。


In [ ]:
vec_jpn = np.array(wv.word_vec('日本'))
print(vec_jpn)
indices = np.argsort(np.linalg.norm(vectors - vec_jpn, axis=1))
print(words[indices[1:11]])

## 4) k-meansで単語ベクトルをクラスタリングする

* かなり時間がかかるので、待つ。
* 得られたクラスタの重心はCSVファイルとして保存しておく。
* クラスタ数は変更してよい。

* クラスタ数の設定



In [ ]:
n_clusters = 100

* k-平均法によるクラスタリングを実行。
 * かなり時間がかかるので、待つ。

In [ ]:
from sklearn.cluster import KMeans

#kmeans = KMeans(n_clusters=n_clusters, random_state=123)
#kmeans.fit(vectors)

* クラスタリングの結果をcsvファイルとして保存。

In [ ]:
#np.savetxt('cluster_centers_{:d}.csv'.format(n_clusters), kmeans.cluster_centers_, delimiter=',')
#centers = kmeans.cluster_centers_

* k-平均法を実行するのではなく、クラスタの重心をファイルから読み込むときは、下のセルを実行。
 * パスは適当に書き換える。


In [ ]:
centers = np.loadtxt('/content/drive/MyDrive/2021Courses/SML/cluster_centers_100.csv', delimiter=',')

In [ ]:
center = centers[10]
indices = np.argsort(np.linalg.norm(vectors - center, axis=1))
print(words[indices[:20]])

# 課題11

* k-平均法で単語ベクトルをクラスタリングする。
* いくつかのクラスタについて、クラスタの重心に近い単語どうしの意味が似ているかどうか、チェックする。
* いくつかのクラスタについて、クラスタの重心から遠い単語がどのような単語になっているかをチェックする。
* 極端にサイズの大きなクラスタや、逆に、極端にサイズの小さなクラスタができていないか、チェックする。
 * そして、そういった極端なサイズのクラスタに属する単語がどのようになっているかを調べる。
* k-平均法以外のクラスタリング手法でも、同じような調査をおこなってみる。




---



## A) 階層的クラスタリングの計算量
* Wikipediaのエントリにあるとおり、階層的クラスタリングは計算量が大きく、規模の大きなデータには向かない。
 * https://en.wikipedia.org/wiki/Hierarchical_clustering
* データ数を$n$とすると、何の工夫もしなければ時間計算量$O(n^3)$。
* なので、階層的クラスタリングは使わなくてよい。



---



## B) `MiniBatchKMeans`を試してみる

* 大規模データを扱うためには、ミニバッチ方式で動くアルゴリズムがあれば、それを使うのが良いだろう。
* 幸い、k-平均法には、ミニバッチ版がある。
* これにより、データをミニバッチとして少しずつ与えて、クラスタリングをすこしずつ進めることができる。
* そこで、ミニバッチ式のクラスタリングのについて、途中経過を調べてみる。

### `partial_fit`の実行を反復してみる

* バッチサイズは1000にしてみる。

In [ ]:
from sklearn.cluster import MiniBatchKMeans

n_clusters = 100
kmeans = MiniBatchKMeans(n_clusters=n_clusters, random_state=123, batch_size=1000, max_iter=1)

* まずは1回、partial_fitを実行。

In [ ]:
kmeans.partial_fit(vectors)

* 適当なクラスタ（ここではインデックスが30のクラスタ）の、重心に近い順に上位20語を表示させてみる。

In [ ]:
center = kmeans.cluster_centers_[30]
indices = np.argsort(np.linalg.norm(vectors - center, axis=1))
print(' '.join(words[indices[:20]]))

* もう1回、partial_fitを実行。

In [ ]:
kmeans.partial_fit(vectors)
center = kmeans.cluster_centers_[30]
indices = np.argsort(np.linalg.norm(vectors - center, axis=1))
print(' '.join(words[indices[:20]]))

* 少しクラスタの様子が変わっている。さらにもう1回、partial_fitを実行。

In [ ]:
kmeans.partial_fit(vectors)
center = kmeans.cluster_centers_[30]
indices = np.argsort(np.linalg.norm(vectors - center, axis=1))
print(' '.join(words[indices[:20]]))

* やはり変化している。10回ループを回してみる。

In [ ]:
for i in range(10):
  kmeans.partial_fit(vectors)
  center = kmeans.cluster_centers_[30]
  indices = np.argsort(np.linalg.norm(vectors - center, axis=1))
  print(' '.join(words[indices[:20]].tolist()))

* さらに10回、ループを回す。
 * ループを回すついでに、所属するクラスタが変化した単語がいくつあったかを、毎回表示させてみる。


In [ ]:
for i in range(10):
  prev_labels = kmeans.labels_
  kmeans.partial_fit(vectors)
  center = kmeans.cluster_centers_[30]
  indices = np.argsort(np.linalg.norm(vectors - center, axis=1))
  print(' '.join(words[indices[:20]].tolist()))
  print(f'{(kmeans.labels_ != prev_labels).sum()}個の単語が別のクラスタへ移った。')

* このあたりで、クラスタの重心を保存しておく。
 * ちなみに、保存したクラスタの重心は、再度同じデータセット上でk-平均法を実行するときに、クラスタリングの初期値として使える。

In [ ]:
np.savetxt('MiniBatchKMeans_cluster_centers_{:d}.csv'.format(n_clusters), kmeans.cluster_centers_, delimiter=',')

* さらに30回ほど回してみる。

In [ ]:
for i in range(30):
  prev_labels = kmeans.labels_
  kmeans.partial_fit(vectors)
  center = kmeans.cluster_centers_[30]
  indices = np.argsort(np.linalg.norm(vectors - center, axis=1))
  print(' '.join(words[indices[:20]].tolist()))
  print(f'{(kmeans.labels_ != prev_labels).sum()}個の単語が別のクラスタへ移った。')

* 本当は、すべての単語がクラスタリングの対象になるまで、ループを回す必要あり。

* おそらく、最低、（単語数）÷（ミニバッチサイズ）回は、回す必要があると思われる。

 * partial_fitメソッドの仕様を、みなさん調べておいてください。

### クラスタのサイズを調べる

* NumPyの配列に、いろいろな値が何回ずつ出てくるかを知るには、unique関数を使うと良い。

In [ ]:
unique, counts = np.unique(kmeans.labels_, return_counts=True)

In [ ]:
unique

In [ ]:
counts

* クラスタのインデックスをキーとし、そのサイズを値とする辞書を作る。

In [ ]:
size_dict = dict(zip(unique, counts))

* 辞書のエントリを、キーではなく値でソートする。

In [ ]:
sorted_clusters = [k for k, v in sorted(size_dict.items(), key=lambda item: item[1], reverse=True)]

In [ ]:
counts[sorted_clusters]

### サイズが最大のクラスタを調べる

In [ ]:
center = kmeans.cluster_centers_[sorted_clusters[0]]
indices = np.argsort(np.linalg.norm(vectors - center, axis=1))
' '.join(words[indices[:100]].tolist())

* あまり統一感がない。クラスタ数100個が少なかった可能性もある。

### サイズが最小のクラスタを調べる

In [ ]:
center = kmeans.cluster_centers_[sorted_clusters[99]]
indices = np.argsort(np.linalg.norm(vectors - center, axis=1))
' '.join(words[indices[:100]].tolist())

* 命令的な表現が多いようだ。

### サイズが中間的なクラスタを調べる

In [ ]:
center = kmeans.cluster_centers_[sorted_clusters[49]]
indices = np.argsort(np.linalg.norm(vectors - center, axis=1))
' '.join(words[indices[:100]].tolist())

* かなり統一感がある。

### 適当なクラスターを調べてみる

In [ ]:
center = kmeans.cluster_centers_[sorted_clusters[39]]
indices = np.argsort(np.linalg.norm(vectors - center, axis=1))
' '.join(words[indices[:100]].tolist())

* スポーツ関係っぽい。

## C) Dask-MLのspectral clusteringを試してみる

* Dask-MLについてはWebサイトを参照（私も詳しくは知りません・・・）。
 * https://ml.dask.org/

* scalableなspectral clustering（ただし近似が入る）が実装されているので、使ってみる。
 * https://ml.dask.org/clustering.html

### Daskをインストール

In [ ]:
! pip install -U tornado
! pip install dask distributed --upgrade
! pip install dask_ml

* 単語ベクトルのデータを取得し直す



In [ ]:
import numpy as np
from gensim.models.word2vec import Word2Vec

model_path = 'word2vec.gensim.model'
model = Word2Vec.load(model_path)
wv = model.wv
words = list()
vectors = list()
for word in wv.vocab:
  words.append(word)
  vectors.append(wv.word_vec(word))
words = np.array(words)
vectors = np.array(vectors)

### spectral clusteringを実行する

In [ ]:
import dask_ml.cluster

n_clusters = 100
clusterer = dask_ml.cluster.SpectralClustering(n_clusters=n_clusters, random_state=123,
                                               n_jobs=-1, n_components=100)

* クラスタリングの実行。
 * しばらく待つ。

In [ ]:
clusterer.fit(vectors)

* ラベル（各データ点がどのクラスタに属するか）の情報のデータ型を調べる。

In [ ]:
labels = clusterer.labels_
type(labels)

* Dask固有のデータ型のようなのでNumpyの配列に変換する。

In [ ]:
labels = labels.compute()
type(labels)

* ラベルをファイルに保存しておく。

In [ ]:
np.save('dask_sc_labels', labels)

* クラスタのサイズ順にクラスタのインデックス（0~99）をソートする。

In [ ]:
unique, counts = np.unique(labels, return_counts=True)
size_dict = dict(zip(unique, counts))
sorted_clusters = [k for k, v in sorted(size_dict.items(), key=lambda item: item[1], reverse=True)]

### クラスタのサイズを調べる

In [ ]:
counts[sorted_clusters]

### サイズが最大のクラスタを調べる

In [ ]:
' '.join(words[labels == sorted_clusters[0]])

### サイズが最小のクラスタを調べる

In [ ]:
' '.join(words[labels == sorted_clusters[99]])

### サイズが中間的なクラスタを調べる

In [ ]:
' '.join(words[labels == sorted_clusters[49]])